In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, CategoricalAccuracy
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

2023-09-13 12:30:13.006236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/theanik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/theanik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/theanik/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/theanik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/theanik/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alrea

True

In [9]:

question_data = pd.read_csv('../data/processed/training_questions.csv', on_bad_lines='skip')

question_label = []
chat_data = {}
for index, row in question_data.iterrows():
    question_label.append(row["label"])
    if row["label"] in chat_data:
        chat_data[row["label"]].append(row["question"])
    else:
        chat_data[row["label"]] = [row["question"]]

In [10]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(question_label)
print(encoded_labels)

[49 49 49 49 49 49 49 49 49 49 49 49 47 47 47 47 47 47 47 47 47 47 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 50 50 50 50  0 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 19 19 19 19 19 19 19 19 19 41 41 41 41 41 41 41 41 41 41 41 41 41 41 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 42 42 42 42 42 42 42 42 42
 42 42 42 42 42 42 42 42  2  2  2  2  2  2  2  2  2  2  1  1  1  6  6  6
  6  6  6  6  6  6  6  5  5  5  5  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3 13 13 13 13 13 13 13

In [11]:
stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


def chat_tokenizer(doc):
    
    words = word_tokenize(doc)
    words = [stemmer.stem(word) for word in words]

    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    print(non_punctuation)
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [12]:
vectorizer = TfidfVectorizer(tokenizer=chat_tokenizer)
tfidf_vectorizer = vectorizer.fit_transform(tuple(question_data['question']))

[('work', 'NN'), ('hour', 'NN'), ('porton', 'NN')]
[('offic', 'JJ'), ('start', 'NN'), ('end', 'NN')]
[('porton', 'JJ'), ('oper', 'IN'), ('hour', 'NN')]
[('offic', 'JJ'), ('open', 'JJ'), ('close', 'JJ')]
[('time', 'NN'), ('frame', 'NN'), ('work', 'NN'), ('hour', 'NN')]
[('work', 'JJ'), ('hour', 'NN'), ('start', 'NN'), ('09:00', 'CD')]
[('work', 'NN'), ('hour', 'NN'), ('end', 'VBN')]
[('work', 'NN'), ('hour', 'NN'), ('start', 'RB')]
[('regular', 'JJ'), ('hour', 'NN'), ('oper', 'NN'), ('porton', 'NN')]
[('could', 'MD'), ('let', 'VB'), ('know', 'VB'), ('typic', 'NN'), ('work', 'NN'), ('hour', 'NN'), ('porton', 'NN')]
[('doe', 'JJ'), ('porton', 'NN'), ('usual', 'JJ'), ('start', 'NN'), ('end', 'VB'), ('workday', 'JJ')]
[('standard', 'JJ'), ('hour', 'NN'), ('porton', 'NN'), ('open', 'JJ')]
[('mani', 'JJ'), ('weekend', 'NN'), ('porton', 'NN')]
[('mani', 'JJ'), ('weekend', 'NN'), ('doe', 'NN'), ('porton', 'NN')]
[('mani', 'JJ'), ('day', 'NN'), ('week', 'NN'), ('porton', 'JJ'), ('open', 'JJ')]
[

/home/theanik/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[('exra', 'JJ'), ('facil', 'NN'), ('avail', 'NN')]
[('addit', 'NN'), ('facil', 'NN'), ('provid', 'VBN')]
[('tell', 'VB'), ('porton', 'NN'), ('benefit', 'NN')]
[('kind', 'NN'), ('fo', 'JJ'), ('benefit', 'NN'), ('provid', 'NN'), ('porton', 'NN')]
[('typic', 'NN'), ('salari', 'NN'), ('rang', 'NN'), ('softwar', 'JJ'), ('engin', 'NN')]
[('typic', 'NN'), ('salari', 'NN'), ('rang', 'NN'), ('softwar', 'NN'), ('develop', 'NN')]
[('porton', 'NN'), ('salari', 'NN'), ('rang', 'NN'), ('softwar', 'JJ'), ('engin', 'NN')]
[('porton', 'NN'), ('salari', 'NN'), ('structur', 'NN'), ('softwar', 'NN'), ('develop', 'NN')]
[('porton', 'NN'), ('salari', 'NN'), ('structur', 'NN')]
[('porton', 'NN'), ('salari', 'NN'), ('rang', 'NN')]
[('could', 'MD'), ('provid', 'VB'), ('inform', 'VB'), ('salari', 'NN'), ('rang', 'NN'), ('softwar', 'JJ'), ('engin', 'NN'), ('porton', 'NN')]
[('salari', 'JJ'), ('structur', 'NN'), ('softwar', 'JJ'), ('develop', 'NN'), ('porton', 'NN')]
[('could', 'MD'), ('share', 'NN'), ('detail', 

In [13]:
# Build the neural network model
# model = Sequential([
#     # Dense(1024, activation='relu', input_dim=tfidf_vectorizer.shape[1]),
#     # Input(shape=(344,)),
#     Dense(512, activation='relu', input_dim=tfidf_vectorizer.shape[1]),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])


model = Sequential([
    # Dense(1024, activation='relu', input_dim=tfidf_vectorizer.shape[1]),
    # Input(shape=(344,)),
    Dense(512, activation='relu', input_dim=tfidf_vectorizer.shape[1]),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])



# Compile the model
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam(learning_rate=0.001), metrics=[SparseCategoricalAccuracy()])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectorizer.toarray(), encoded_labels, test_size=0.02, random_state=42)
model.fit(X_train, y_train, epochs=1000, batch_size=32)




Epoch 1/1000


/home/theanik/anaconda3/lib/python3.11/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


26/26 [==============================] - 3s 37ms/step - loss: 3.8508 - sparse_categorical_accuracy: 0.0755
Epoch 2/1000
26/26 [==============================] - 1s 32ms/step - loss: 3.4359 - sparse_categorical_accuracy: 0.1535
Epoch 3/1000
26/26 [==============================] - 1s 41ms/step - loss: 2.8325 - sparse_categorical_accuracy: 0.2946
Epoch 4/1000
26/26 [==============================] - 1s 44ms/step - loss: 2.2789 - sparse_categorical_accuracy: 0.4592
Epoch 5/1000
26/26 [==============================] - 1s 28ms/step - loss: 1.7835 - sparse_categorical_accuracy: 0.6287
Epoch 6/1000
26/26 [==============================] - 1s 25ms/step - loss: 1.3349 - sparse_categorical_accuracy: 0.7723
Epoch 7/1000
26/26 [==============================] - 1s 41ms/step - loss: 1.0097 - sparse_categorical_accuracy: 0.8243
Epoch 8/1000
26/26 [==============================] - 1s 41ms/step - loss: 0.7472 - sparse_categorical_accuracy: 0.8998
Epoch 9/1000
26/26 [==============================] -

In [14]:
# # Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f} \n \n')

print(f'Test accuracy: {accuracy:.4f} \n \n')


print(f'Model Summary : ')

model.summary()


1/1 [==============================] - 0s 295ms/step - loss: 1.4969 - sparse_categorical_accuracy: 0.9412
Test loss: 1.4969 
 

Test accuracy: 0.9412 
 

Model Summary : 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               220160    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 51)                6579      
                                                                 
Total params: 2

In [15]:
example_question = ["how are you"]
example_question_vectorized = vectorizer.transform(example_question)
predicted_label = model.predict(example_question_vectorized.toarray())
print(predicted_label)
predicted_label_index = np.argmax(predicted_label)
print(predicted_label_index)
predicted_label_name = label_encoder.classes_[predicted_label_index]

threshold = 0.90
binary_predictions = (predicted_label >= threshold).astype(int)
print(binary_predictions.sum())
if binary_predictions.sum():
    print(predicted_label_name)
else:
    print("Sorry I can't understnand")

[]


1/1 [==============================] - 0s 151ms/step
[[0.00615802 0.07405332 0.05799013 0.06513879 0.02456489 0.0044165
  0.01361876 0.00566834 0.00261006 0.01298035 0.13903683 0.07227915
  0.0032467  0.05142091 0.00510285 0.02917519 0.00732766 0.02123826
  0.0079091  0.00537272 0.05231342 0.0180583  0.0028038  0.0089119
  0.00465575 0.01866464 0.00665396 0.00539146 0.00240737 0.0131031
  0.00073374 0.00400453 0.00137905 0.00257182 0.00145549 0.00527397
  0.0185338  0.03857195 0.00558192 0.0297972  0.00845157 0.00502875
  0.01702659 0.0150705  0.02465955 0.04546624 0.01232372 0.00462711
  0.0057935  0.00778311 0.00359377]]
10
0
Sorry I can't understnand
